# Encodage du texte

In [1]:
import numpy as np

# supposant nous avons 4 textes (les lignes)
# pour chaque texte (colonne 0), on accorde une classe (colonne 1)
data = np.array([
    ["This is the first document.", 1],
    ["This document is the second document.", 0],
    ["this is the third one not the first nor the third.", 1],
    ["Is this the first document or is is another document?", 0]
])

data

array([['This is the first document.', '1'],
       ['This document is the second document.', '0'],
       ['this is the third one not the first nor the third.', '1'],
       ['Is this the first document or is is another document?', '0']],
      dtype='<U53')

In [2]:
# TF : term frequency
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
X = count_vectorizer.fit_transform(data[:, 0]) #appliquer sur toutes les lignes, colonne 0

# Afficher les différents terms
print(count_vectorizer.get_feature_names())

# Afficher la matrice (document X term)
X.toarray()

['another', 'document', 'first', 'is', 'nor', 'not', 'one', 'or', 'second', 'the', 'third', 'this']


array([[0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1],
       [0, 2, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1],
       [0, 0, 1, 1, 1, 1, 1, 0, 0, 3, 2, 1],
       [1, 2, 1, 3, 0, 0, 0, 1, 0, 1, 0, 1]])

In [3]:
from sklearn.metrics.pairwise import cosine_similarity

# la fonction accepte deux vecteurs X et Y (pour calculer la similarité entre les deux)
# Si on fourni X comme une matrice, sans fournir Y
# la fonction va calculer la cosinus entre chaque ligne
# Donc, on aura une matrice (document X document)
Sim = cosine_similarity(X)

# Bien sûr, le diagonale aura 1 puisque c'est la similarité du texte avec lui-même
Sim

array([[1.        , 0.79056942, 0.61558701, 0.84327404],
       [0.79056942, 1.        , 0.40555355, 0.75      ],
       [0.61558701, 0.40555355, 1.        , 0.43259046],
       [0.84327404, 0.75      , 0.43259046, 1.        ]])

In [4]:
# Pour calculer la moyenne des similarité de chaque document avec les autres
# On fait la somme sur les lignes ou les colonne (la matrice est symétrique : égale à son transposé)
# On dimunie la somme par 1 (la similarité du document avec lui-même)
# ensuite, on divise sur le nombre des lignes (documents)
Moy = (Sim.sum(axis=0) - 1)/Sim.shape[0]

Moy

array([0.56235762, 0.48653074, 0.36343276, 0.50646612])

In [5]:
# Supposant, on veut garder seulement les documents avec une similarité moyenne >= 0.5
Garder = Moy >= 0.5

Garder

array([ True, False, False,  True])

In [6]:
# Extraire les documents à garder
# Sélectionner les lignes avec True et la colonne 0
Documents_pertinents = data[Garder, 0]

Documents_pertinents

array(['This is the first document.',
       'Is this the first document or is is another document?'],
      dtype='<U53')

In [7]:
# TF-IDF
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
from sklearn.feature_extraction.text import TfidfVectorizer

# Ici, je vais fournir une liste des mots vides
mots_vides = {"not", "is", "are", "this", "that", "or", "and", "the", "nor"}
tfidf_vectorizer = TfidfVectorizer(stop_words=mots_vides)
X_tfidf = tfidf_vectorizer.fit_transform(data[:, 0]) #appliquer sur toutes les lignes, colonne 0

# Afficher les différents terms
print(tfidf_vectorizer.get_feature_names())

# Afficher la matrice (document X term)
X_tfidf.toarray()

['another', 'document', 'first', 'one', 'second', 'third']


array([[0.        , 0.70710678, 0.70710678, 0.        , 0.        ,
        0.        ],
       [0.        , 0.78722298, 0.        , 0.        , 0.61666846,
        0.        ],
       [0.        , 0.        , 0.27448674, 0.43003652, 0.        ,
        0.86007303],
       [0.57381765, 0.73252075, 0.36626037, 0.        , 0.        ,
        0.        ]])

## Word2Vec en iutilisant Gensim

In [22]:
# ici, un tableau de phrases, où chaque phrase contient des mots
phrases = [[mot for mot in phrase.split()] for phrase in data[:, 0]]

phrases

[['This', 'is', 'the', 'first', 'document.'],
 ['This', 'document', 'is', 'the', 'second', 'document.'],
 ['this',
  'is',
  'the',
  'third',
  'one',
  'not',
  'the',
  'first',
  'nor',
  'the',
  'third.'],
 ['Is',
  'this',
  'the',
  'first',
  'document',
  'or',
  'is',
  'is',
  'another',
  'document?']]

In [31]:
from gensim.models import Word2Vec

#un modèle avec un vecteur de 2, une fenêtre de 3 et 20 itérations
model_w2v= Word2Vec(sentences=phrases, vector_size=2, window=3, epochs=20, min_count=1)
word_vectors = model_w2v.wv

print ("vocabulaire = " + str(word_vectors.key_to_index))
print("Le embedding de 'first' = " + str(word_vectors["first"]))
print("Le embedding de 'document' = " + str(word_vectors.get_vector("document")))
print("cos('first', 'document') = " + str(word_vectors.distance("first","document")))

vocabulaire = {'the': 0, 'is': 1, 'first': 2, 'This': 3, 'this': 4, 'document.': 5, 'document': 6, 'second': 7, 'document?': 8, 'another': 9, 'one': 10, 'not': 11, 'nor': 12, 'third.': 13, 'Is': 14, 'or': 15, 'third': 16}
Le embedding de 'first' = [-0.46529216 -0.35571396]
Le embedding de 'document' = [-0.22693975  0.3276365 ]
cos('first', 'document') = 1.0469160676002502


In [32]:
# pour sauvegarder le word2vec : il ne faut pas entrainer à chaque fois
model_w2v.save("word2vec.model")

In [35]:
# récupérer un modèle sauvegardé
model2 = Word2Vec.load("word2vec.model")

#tester le modèle 2 avec les mots les plus proches à un autre
# les 5 mots les plus similaires
mot = "another"
sim_mot = []
if mot in model2.wv: # tester si le mot existe dans la vocabulaire
    sim_mot = model2.wv.most_similar(mot, topn=5)
sim_mot

[('first', 0.980423629283905),
 ('this', 0.978571891784668),
 ('Is', 0.9327181577682495),
 ('second', 0.8803404569625854),
 ('or', 0.7467302083969116)]

In [ ]:
# Pour plus d'info sur Gensim 
# https://radimrehurek.com/gensim/models/word2vec.html

#Il y a aussi , doc2vec 
# https://radimrehurek.com/gensim/models/doc2vec.html